<a href="https://colab.research.google.com/github/zdwhite/Thinkful-Unit-3/blob/master/Challenge_If_a_tree_falls_in_the_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Build the best decision tree you can.

OKAY DAD! For the interest of time let's use the previous data set and test some run times!

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import timeit


In [2]:
y2015 = pd.read_csv(
    'https://www.dropbox.com/s/0so14yudedjmm5m/LoanStats3d.csv?dl=1',
    skipinitialspace=True,
    header=1)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
####### REMEMBER THIS SHIT could be REALLY useful in the future
categorical = y2015.select_dtypes(include=['object'])
for i in categorical:
    column = categorical[i]
    print(i,column.nunique())
    #print()
#### 

In [0]:
# Convert ID and Interest Rate to numeric.
y2015['id'] = pd.to_numeric(y2015['id'], errors='coerce')
y2015['int_rate'] = pd.to_numeric(y2015['int_rate'].str.strip('%'), errors='coerce')

# Drop other columns with many unique variables
y2015.drop(['url', 'emp_title', 'zip_code', 'earliest_cr_line', 'revol_util',
            'sub_grade', 'addr_state', 'desc'], 1, inplace=True)

In [0]:
# Remove two summary rows at the end that don't actually contain data.
y2015 = y2015[:-2]

In [0]:
# Create X and Y then generate our T.T.S reports not to be confused with TPS reports
from sklearn.model_selection import train_test_split

X = y2015.drop(['loan_status','id','member_id'], 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

#Split the data into training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

## The Best Models, you'll lovem, they're great, just the best, trust me

In [14]:
from sklearn import ensemble
from sklearn.model_selection import cross_val_score

rfc = ensemble.RandomForestClassifier(n_estimators=5)
start = timeit.default_timer()
TBM = rfc.fit(X_train, Y_train)
stop = timeit.default_timer()
print(TBM.score(X_test,Y_test))
print("--- %s seconds ---" % (stop - start))

0.9788741759598761
--- 10.738114627001778 seconds ---


## These models? They're the worst!

In [15]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
start = timeit.default_timer()
TWM = dt.fit(X_train, Y_train)
stop = timeit.default_timer()
print(TWM.score(X_test,Y_test))
print("--- %s seconds ---" % (stop - start))


0.9652145828979615
--- 40.75740994499938 seconds ---


See my [github](https://github.com/zdwhite/Thinkful-Unit-3/blob/master/DRILL_Random_forest.ipynb) on the previous drill for how I came up with the Principle Components for this next run.

In [0]:
from matplotlib.mlab import PCA as mlabPCA
from sklearn import preprocessing
from sklearn.decomposition import PCA 

In [23]:
# Let's condense things down to 30 dimmensions 
sklearn_pca = PCA(n_components=10)

## use a standard scalar for our data we'll drop the dependant variable and the id columns

X = y2015.drop(['loan_status','id','member_id'],1)
X = pd.get_dummies(X)
X = X.dropna(axis=1)

data_scaled = pd.DataFrame(preprocessing.scale(X),columns = X.columns)

Y_sklearn = sklearn_pca.fit_transform(data_scaled)

principalDf = pd.DataFrame(data = Y_sklearn)
FinalDf = pd.concat([principalDf, y2015['loan_status']], axis = 1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype uint8, float64 were all converted to float64 by the scale function.
  if __name__ == '__main__':


In [0]:
X = FinalDf.drop(['loan_status'], 1)
Y = FinalDf['loan_status']
#Split the data into training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)


In [25]:
rfc = ensemble.RandomForestClassifier(n_estimators=5)
start = timeit.default_timer()
TBM = rfc.fit(X_train, Y_train)
stop = timeit.default_timer()
print(TBM.score(X_test,Y_test))
print("--- %s seconds ---" % (stop - start))

0.934485248019454
--- 9.605930215002445 seconds ---


After only keeping  10 Principal components we see our accuracy dropped ~4%  for a marginal gain in runtime efficiency. Let's drop some other stuff and see what happens

In [0]:
# Below is a list of features that will be removed that are all highly correlated with one another again see my github for the previous assignment on how I ffound these

features = ['num_actv_rev_tl',
 'total_il_high_credit_limit',
 'open_rv_12m',
 'id',
 'num_bc_tl',
 'funded_amnt',
 'out_prncp',
 'loan_amnt',
 'revol_bal',
 'installment',
 'num_actv_bc_tl',
 'open_rv_24m',
 'open_acc',
 'tot_cur_bal',
 'total_bal_il',
 'bc_util',
 'member_id',
 'mths_since_recent_bc_dlq',
 'mths_since_last_delinq',
 'num_rev_tl_bal_gt_0',
 'total_rec_prncp',
 'total_pymnt',
 'num_sats',
 'bc_open_to_buy',
 'last_pymnt_amnt',
 'loan_status',
 'recoveries','loan_status']

In [0]:
# Create X and Y then generate our T.T.S reports not to be confused with TPS reports
from sklearn.model_selection import train_test_split

X = y2015.drop(features, 1)
Y = y2015['loan_status']
X = pd.get_dummies(X)
X = X.dropna(axis=1)

#Split the data into training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X,Y)

In [29]:
rfc = ensemble.RandomForestClassifier(n_estimators=5)
start = timeit.default_timer()
TBM = rfc.fit(X_train, Y_train)
stop = timeit.default_timer()
print(TBM.score(X_test,Y_test))
print("--- %s seconds ---" % (stop - start))

0.9733552444098258
--- 8.079173739999533 seconds ---


Test the single tree model with the paired down data

In [30]:
dt = DecisionTreeClassifier()
start = timeit.default_timer()
TWM = dt.fit(X_train, Y_train)
stop = timeit.default_timer()
print(TWM.score(X_test,Y_test))
print("--- %s seconds ---" % (stop - start))


0.9642741797594848
--- 26.202839564000897 seconds ---


Looks like we have a winner  
**0.9733552444098258   --- 8.079173739999533 seconds ---** 

as compared to our Random Forest classifier with all the features

0.9788741759598761   --- 10.738114627001778 seconds ---